# Отток клиентов

Из банка стали уходить клиенты. Требуется спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Требуется построить модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Выгрузка библиотек и данных для работы

In [2]:
# Выгрузка стандартных билиотек
import warnings
import random

# Выгрузка сторонних библиотек
import pandas as pd

#Выгрузка модулей библиотек для машинного обучения
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

# Объявление константы для "фиксации случайности"
rnd_st = 12345

# Настройка игнорирования ошибок
warnings.filterwarnings("ignore")
random.seed(rnd_st)

Выгрузим данные в переменную df.

In [3]:
try:
    df = pd.read_csv('Churn.csv')
except:
    df = pd.read_csv('/datasets/Churn.csv')

Так как в данных есть столбец, отражающий номер строки, сразу его удалим, так как он нам не понадобится

In [4]:
df = df.drop('RowNumber', axis=1)

Просмотрим общую информацию о данном датафрейме

In [29]:
df.sort_values(by='tenure', ascending=False)

,credit_score,age,tenure,balance,num_of_products,has_credit_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
8645,636,20,10.0,124266.86,1,0,0,100566.81,0,0,0,0
1953,623,21,10.0,0.00,2,0,1,135851.30,0,0,0,1
410,709,23,10.0,0.00,2,0,0,129590.18,0,0,1,1
5884,552,38,10.0,132271.12,2,1,1,46562.02,0,1,0,1
408,668,37,10.0,152958.29,2,1,1,159585.61,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4722,648,32,0.0,0.00,1,0,1,117323.31,0,0,0,1
9523,660,32,0.0,114668.89,1,1,0,84605.00,0,0,0,0
6878,651,35,0.0,181821.96,2,0,1,36923.67,1,0,0,1
427,702,45,0.0,80793.58,1,1,1,27474.81,0,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           9091 non-null   float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1015.8+ KB


In [7]:
df.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Описание того, что мы имеем (Значения столбцов, количественные/категориальные данные, пропуски, типы данных)

### Вывод
##### Общая информация
Были рассмотрены данные, получена общая информация:
Всего строк 10000, столбцов 13, среди них с целевым признаком считается столбец Excited.
##### Был проанализирован каждый столбец:

|  №  |  Наименование   |                        Описание                         |               Тип данных                |
|:---:|:---------------:|:-------------------------------------------------------:|:---------------------------------------:|
|  1  |   CustomerId    |         уникальный идентификатор <br/> клиента          | Количественный <br/> и <br/> дискретный |
|  2  |     Surname     |                         фамилия                         |             Категориальный              |
|  3  |   CreditScore   |                    кредитный рейтинг                    | Количественный <br/> и <br/> дискретный |
|  4  |    Geography    |                    страна проживания                    |             Категориальный              |
|  5  |     Gender      |                           пол                           |             Категориальный              |
|  6  |      Age        |                         возраст                         | Количественный <br/> и <br/> дискретный |
|  7  |     Tenure      |    сколько лет человек <br/> является клиентом банка    | Количественный <br/>и <br/> непрерывный |
|  8  |     Balance     |                     баланс на счёте                     | Количественный <br/>и <br/> непрерывный |
|  9  |  NumOfProducts  | количество продуктов банка, <br/> используемых клиентом | Количественный <br/> и <br/> дискретный |
| 10  |    HasCrCard    |              наличие <br/> кредитной карты              |             Категориальный              |
| 11  | IsActiveMember  |                   активность клиента                    |             Категориальный              |
| 12  | EstimatedSalary |              предполагаемая <br/> зарплата              |       Количественый и непрерывный       |
| 13  |     Exited      |                факт ухода <br/> клиента                 |             Категориальный              |
В столбце Tenure были обнаружены строки с пропущенными значениями, которые потребуется заполнить

# 2. Подготовка данных

### 2.0 Перед тем, как мы будем заниматься подготовкой данных для дальнейшего обучения, стоило бы перевести наименования столбцов в корректный вид:

In [8]:
df.columns = ['customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_credit_card', 'is_active_member', 'estimated_salary', 'exited']

Рассмотрим проблемные столбцы датафрейма:

#### Столбцы geography и gender

Столбцы категориальные, пропусков не обнаружено. Рассмотрим существующие уникальные значения

In [9]:
df['geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [10]:
df['gender'].unique()

array(['Female', 'Male'], dtype=object)

Для перевода столбца geography из категориального типа в подходящий для машинного обучения вид следует использовать технику прямого кодирования **OHE** (***One-Hot Encoding***), так как значения не отражают порядок или величину (если бы отражали, было бы уместнее использовать порядковое кодирование **OE** (***Ordinal Encoding***). Воспользуемся методом get_dummies из библиотеки pandas с параметром drop_first=True (этот параметр позволяет удалить первый, легко воспроизводимый из двух других, столбец.
Для перевода столбца gender уместнее конвертировать каждый пол в цифровой формат, к примеру Female: 0, а Male: 1. Это можно реализовать также через метод get_dummies.

In [11]:
df = pd.get_dummies(df, columns=['geography', 'gender'], prefix=['geography', 'gender'], drop_first=True)
for i in ['geography_Germany','geography_Spain','gender_Male']:
    df[i] = df[i].astype(int)

Перевод столбцов с категориальными данными успешно завершнен.

#### Столбец tenure
В столбце обнаружены пропущенные значения, предлагается заполнить их случайными значениями в диапазоне от 0 до 10, чтобы не портить общее распределение данных

In [12]:
df['tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: tenure, dtype: float64

In [13]:
df['tenure'] = df["tenure"].fillna(random.randint(0,10))

In [14]:
df['tenure'].describe()

count    10000.000000
mean         5.088800
std          2.775011
min          0.000000
25%          3.000000
50%          5.000000
75%          7.000000
max         10.000000
Name: tenure, dtype: float64

После заполнения пропусков среднее арифметическое практически не изменилось, медиана осталась неизменной.

### Разделим данные на признаки и целевые признаки (features и target)

In [15]:
df = df.drop(['customer_id','surname'], axis=1)
features = df.drop(['exited'], axis=1)
target = df['exited']

### Разделение данных на тренировочную, валидационную и тестовую выборки

Разделим данные с помощью функции train_test_split библиотеки fast_ml

In [16]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, train_size=0.6, random_state=rnd_st, stratify=target)

In [17]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, train_size=0.5, random_state=rnd_st, stratify=target_valid)

Проверим размер выборок

In [18]:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(6000, 11)
(6000,)
(2000, 11)
(2000,)
(2000, 11)
(2000,)


In [19]:
df

,credit_score,age,tenure,balance,num_of_products,has_credit_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Вывод: Данные готовы к обучению и дальнейшему масштабированию.

### Масштабирование признаков

Выделим те столбцы, которые требуется масштабировать

In [20]:
numeric = ['credit_score',
           'age',
           'tenure',
           'balance',
           'estimated_salary'
           ]

Масштабируем все полученные features

In [21]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
for ftrs in [features_train, features_valid, features_test]:
    ftrs[numeric] = scaler.transform(ftrs[numeric])

## Обучение моделей без балансировки классов

Обучим три разных вида моделей, для оценки будем использовать метрики f1 и ROC-AUC

In [22]:
models = {'name':[], 'model':[],'class_weight':[],'f1':[],'auc_roc':[]}

In [23]:
def add_model(name,model,class_weight,f1,auc_roc):
    models['name'].append(name)
    models['model'].append(model)
    models['class_weight'].append(class_weight)
    models['f1'].append(f1)
    models['auc_roc'].append(auc_roc)

### Линейная регрессия

In [24]:
model = LogisticRegression(random_state=rnd_st, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'f1 = {f1_score(target_valid, predicted_valid)},'
      f' AUC-ROC = {roc_auc_score(target_valid, probabilities_one_valid)}')

add_model('Логистическая регрессия без балансировки классов', model, 'No', f1_score(target_valid, predicted_valid), roc_auc_score(target_valid, probabilities_one_valid))

f1 = 0.3076923076923077, AUC-ROC = 0.7874374938417579


### Решающее дерево

In [25]:
best_model = None
best_f1 = 0
best_auc_roc = 0
for depth in range(1, 6):
    for leaf in range(1,6,1):
                model = DecisionTreeClassifier(max_depth=depth,
                                               min_samples_leaf=leaf,
                                               random_state=rnd_st)
                model.fit(features_train, target_train)
                predicted_valid = model.predict(features_valid)
                f1_v = f1_score(target_valid, predicted_valid)
                probabilities_valid = model.predict_proba(features_valid)
                probabilities_one_valid = probabilities_valid[:, 1]
                auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
                print(f"max_depth= {depth}; min_samples_leaf = {leaf}, f1_v = {f1_v}, roc_auc = {auc_roc_of_model}")
                if f1_v > best_f1:
                    best_f1 = f1_v
                    best_model = model
                    best_auc_roc = auc_roc_of_model
print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Решающее дерево без балансировки классов', best_model, 'No', best_f1, best_auc_roc)

max_depth= 1; min_samples_leaf = 1, f1_v = 0.0, roc_auc = 0.697507143560942
max_depth= 1; min_samples_leaf = 2, f1_v = 0.0, roc_auc = 0.697507143560942
max_depth= 1; min_samples_leaf = 3, f1_v = 0.0, roc_auc = 0.697507143560942
max_depth= 1; min_samples_leaf = 4, f1_v = 0.0, roc_auc = 0.697507143560942
max_depth= 1; min_samples_leaf = 5, f1_v = 0.0, roc_auc = 0.697507143560942
max_depth= 2; min_samples_leaf = 1, f1_v = 0.5203488372093023, roc_auc = 0.7539597497290373
max_depth= 2; min_samples_leaf = 2, f1_v = 0.5203488372093023, roc_auc = 0.7539597497290373
max_depth= 2; min_samples_leaf = 3, f1_v = 0.5203488372093023, roc_auc = 0.7539597497290373
max_depth= 2; min_samples_leaf = 4, f1_v = 0.5203488372093023, roc_auc = 0.7539597497290373
max_depth= 2; min_samples_leaf = 5, f1_v = 0.5203488372093023, roc_auc = 0.7539597497290373
max_depth= 3; min_samples_leaf = 1, f1_v = 0.5375722543352601, roc_auc = 0.8032503202285939
max_depth= 3; min_samples_leaf = 2, f1_v = 0.5375722543352601, roc_a

### Случайный лес

In [26]:
best_model = None
best_f1 = 0
best_auc_roc = 0

for est in [20,50,100]:
    for depth in range(1, 21):
        for leaf in range(1,21):
            model = RandomForestClassifier(n_estimators=est,
                                           max_depth=depth,
                                           min_samples_leaf = leaf,
                                           random_state=rnd_st)
            model.fit(features_train, target_train)
            predicted_valid = model.predict(features_valid)
            f1_v = f1_score(target_valid, predicted_valid)
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
            print(f"n_estimators = {est}, max_depth = {depth}, min_samples_leaf = {leaf}")
            print(f"f1_v = {f1_v}, auc_roc = {auc_roc_of_model}")
            if f1_v > best_f1:
                best_f1 = f1_v
                best_model = model
                best_auc_roc = auc_roc_of_model

print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Случайный лес без балансировки классов', best_model, 'No', best_f1, best_auc_roc)

n_estimators = 20, max_depth = 1, min_samples_leaf = 1
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 2
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 3
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 4
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 5
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 6
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 7
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 8
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 9
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_samples_leaf = 10
f1_v = 0.0, auc_roc = 0.8047152120898611
n_estimators = 20, max_depth = 1, min_s

KeyboardInterrupt: 

Вывод: Лучше всего показала себя модель случайного леса: Ее значение f1 и auc_roc составляет примерно 0.6288 и 0.865 соответственно

### Обучение моделей с учетом балансировки классов

Обучим модели с параметром class_weight='balabced'

### Линейная регрессия

In [ ]:
model = LogisticRegression(random_state=rnd_st, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'f1 = {f1_score(target_valid, predicted_valid)},'
      f' AUC-ROC = {roc_auc_score(target_valid, probabilities_one_valid)}')
add_model('Логистическая регрессия с учетом балансировки класса', model, 'balanced', f1_score(target_valid, predicted_valid), roc_auc_score(target_valid, probabilities_one_valid))

### Решающее дерево

In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0

for depth in range(1, 6):
    for leaf in range(1,6,1):
        model = DecisionTreeClassifier(max_depth=depth,
                                       min_samples_leaf=leaf,
                                       class_weight='balanced',
                                       random_state=rnd_st)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1_v = f1_score(target_valid, predicted_valid)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
        print(f"max_depth= {depth}; min_samples_leaf = {leaf}, f1_v = {f1_v}, roc_auc = {auc_roc_of_model}")
        if f1_v > best_f1:
            best_f1 = f1_v
            best_model = model
            best_auc_roc = auc_roc_of_model

print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Решающее дерево с учетом балансировки класса', best_model, 'balanced', best_f1, best_auc_roc)

### Случайный лес

In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину
for est in [20,50,100]:
    for depth in range(1, 21):
        for leaf in range(1,21):
            model = RandomForestClassifier(n_estimators=est,
                                           max_depth=depth,
                                           min_samples_leaf = leaf,
                                           random_state=rnd_st,
                                           class_weight='balanced')
            model.fit(features_train, target_train)
            predicted_valid = model.predict(features_valid)
            f1_v = f1_score(target_valid, predicted_valid)
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
            print(f"n_estimators = {est}, max_depth = {depth}, min_samples_leaf = {leaf}")
            print(f"f1_v = {f1_v}, auc_roc = {auc_roc_of_model}")
            if f1_v > best_f1:
                best_f1 = f1_v
                best_model = model
                best_auc_roc = auc_roc_of_model

print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Случайный лес с учетом балансировки класса', best_model, 'balanced', best_f1, best_auc_roc)

Вывод: Лучшей моделью машинного обучения также оказалась модель деревьев решений: значения f1 и auc_roc равны примерно 0.658 и 0.8716 соответственно. Результаты оказались немного лучше, чем результаты, полученные без использования параметра балансировки классов class_weight='balanced'

## Исследование дисбаланса классов

Оценим соотношение признаков и целевых признаков

In [ ]:
target_train.value_counts()

Видно, что значений класса 1 примерно в 4 раза меньше, чем значений класса 0.

In [ ]:
features_train.shape

Напишем функцию, которая позволит произвести апсемплинг данных для уравновешивания классов.

In [ ]:
def upsample(repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(4)

In [ ]:
target_upsampled.value_counts()

Видно, что полученные данные после апсемплинга примерно уравновешены в соотношении 1:1.

In [ ]:
features_upsampled.shape

## Обучение моделей с учетом апсемплинга

Обучим модели с учетом апсемплинга тренировочных данных

### Логистическая регрессия

In [ ]:
model = LogisticRegression(random_state=rnd_st, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'f1 = {f1_score(target_valid, predicted_valid)},'
      f' AUC-ROC = {roc_auc_score(target_valid, probabilities_one_valid)}')
add_model('Логистическая регрессия с учетом апсемплинга', model, 'No', f1_score(target_valid, predicted_valid), roc_auc_score(target_valid, probabilities_one_valid))

### Решающее дерево

In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0

for depth in range(1, 6):
    for leaf in range(1,6,1):
        model = DecisionTreeClassifier(max_depth=depth,
                                       min_samples_leaf=leaf,
                                       random_state=rnd_st)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        f1_v = f1_score(target_valid, predicted_valid)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
        print(f"max_depth= {depth}; min_samples_leaf = {leaf}, f1_v = {f1_v}, roc_auc = {auc_roc_of_model}")
        if f1_v > best_f1:
            best_f1 = f1_v
            best_model = model
            best_auc_roc = auc_roc_of_model

print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Решающее дерево с учетом апсемплинга', best_model, 'No', best_f1, best_auc_roc)

### Случайный лес

In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину
for est in [20,50,100]:
    for depth in range(1, 21):
        for leaf in range(1,21):
            model = RandomForestClassifier(n_estimators=est,
                                           max_depth=depth,
                                           min_samples_leaf=leaf,
                                           random_state=rnd_st)
            model.fit(features_upsampled, target_upsampled)
            predicted_valid = model.predict(features_valid)
            f1_v = f1_score(target_valid, predicted_valid)
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
            print(f"n_estimators = {est}, max_depth = {depth}, min_samples_leaf = {leaf}")
            print(f"f1_v = {f1_v}, auc_roc = {auc_roc_of_model}")
            if f1_v > best_f1:
                best_f1 = f1_v
                best_model = model
                best_auc_roc = auc_roc_of_model

print()
print('Лучшая модель:', best_model)
print('f1 лучшей модели:', best_f1)
print('AUC-ROC лучшей модели:', best_auc_roc)
add_model('Случайный лес с учетом апсемплинга', best_model, 'No', best_f1, best_auc_roc)

Вывод: Лучшая модель среди рассмотренных в данном разделе – случайный лес. Полученные значения метрик f1 и auc_roc: 0.655 и 0.874 соответственно.

Рассмотрим сформированный датафрейм и выберем лучшую модель среди всех рассмотренных

In [ ]:
dd = pd.DataFrame(models)

In [ ]:
dd.sort_values('f1')

Лучшей моделью оказался случайный лес с методом class_weight='balanced' со следующими гиперпараметрами: max_depth=20, min_samples_leaf=6, n_estimators=50. Обучим модель еще раз и протестируем ее на всех выборках с помощью метрик f1 и auc_roc

In [ ]:
model = RandomForestClassifier(class_weight='balanced',
                               max_depth=20,
                               min_samples_leaf=6,
                               n_estimators=50,
                               random_state=12345)
model.fit(features_train, target_train)
predicted_train = model.predict(features_train)
predicted_valid = model.predict(features_valid)
predicted_test = model.predict(features_test)
f1_train = f1_score(target_train, predicted_train)
f1_valid = f1_score(target_valid, predicted_valid)
f1_test = f1_score(target_test, predicted_test)
probabilities_train = model.predict_proba(features_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_test = model.predict_proba(features_test)
probabilities_one_train = probabilities_train[:, 1]
probabilities_one_valid = probabilities_valid[:, 1]
probabilities_one_test = probabilities_test[:, 1]

auc_roc_train = roc_auc_score(target_train, probabilities_one_train)
auc_roc_valid = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc_test = roc_auc_score(target_test, probabilities_one_test)
print(f'f1 на тренировочной выборке: {f1_train}')
print(f'f1 на валидационной выборке: {f1_valid}')
print(f'f1 на тестовой выборке: {f1_test}')
print('\n')
print(f'auc_roc на тренировочной выборке: {auc_roc_train}')
print(f'auc_roc на валидационной выборке: {auc_roc_valid}')
print(f'auc_roc на тестовой выборке: {auc_roc_test}')


# Вывод

На протяжение работы были построены и обучены модели трех типов: решающее дерево, случайный лес и логистическая регрессия. Был произведен подбор гиперпараметров моделей и выбрана лучшая модель для каждого типа.
Были изучены два метода решения проблемы дисбаланса классов: добавление гиперпараметра class_weight='balanced', и апсемплинг данных.
Лучшей моделью оказалась модель случайного леса, обученная на тренировочной выборке с добавлением гиперпараметра class_weight='balanced'. Полученные результаты:
f1 на тренировочной выборке: 0.81; f1 на валидационной выборке: 0.658; f1 на тестовой выборке: 0.6148,
auc_roc на тренировочной выборке: 0.97; auc_roc на валидационной выборке: 0.87; auc_roc на тестовой выборке: 0.855

[x] Jupyter Notebook открыт
[x] Весь код выполняется без ошибок
[x] Ячейки с кодом расположены в порядке исполнения
[x] Выполнен шаг 1: данные подготовлены
[x] Выполнен шаг 2: задача исследована
[x] Исследован баланс классов
[x] Изучены модели без учёта дисбаланса
[x] Написаны выводы по результатам исследования
[x] Выполнен шаг 3: учтён дисбаланс
[x] Применено несколько способов борьбы с дисбалансом
[x] Написаны выводы по результатам исследования
[x] Выполнен шаг 4: проведено тестирование
[x] Удалось достичь F1-меры не менее 0.59
[x] Исследована метрика AUC-ROC